In [32]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import torch

torch.set_float32_matmul_precision('high')

test_data = pd.read_csv('data/test_preprocessed.csv')
test_image_path = 'data/test_images/'

test_data['Images'] = test_data['id'].apply(lambda x: os.path.join(test_image_path, str(x) + '.jpeg'))


In [33]:
from autogluon.multimodal import MultiModalPredictor

predictor1 = MultiModalPredictor.load(".\\AutoluonModel\\feature1")
predictor2 = MultiModalPredictor.load(".\\AutoluonModel\\feature2")
predictor3 = MultiModalPredictor.load(".\\AutoluonModel\\feature3")
predictor4 = MultiModalPredictor.load(".\\AutoluonModel\\feature4") # remove id doesn't help
predictor5 = MultiModalPredictor.load(".\\AutoluonModel\\feature5") # remove id doesn't help
predictor6 = MultiModalPredictor.load(".\\AutoluonModel\\idremovedfeature6")

Load pretrained checkpoint: c:\Users\YZM\Desktop\for fun\pytorch\CS480\project\ML-Plant-Properties-Predicion\AutoluonModel\feature1\model.ckpt
Load pretrained checkpoint: c:\Users\YZM\Desktop\for fun\pytorch\CS480\project\ML-Plant-Properties-Predicion\AutoluonModel\feature2\model.ckpt
Load pretrained checkpoint: c:\Users\YZM\Desktop\for fun\pytorch\CS480\project\ML-Plant-Properties-Predicion\AutoluonModel\feature3\model.ckpt
Load pretrained checkpoint: c:\Users\YZM\Desktop\for fun\pytorch\CS480\project\ML-Plant-Properties-Predicion\AutoluonModel\feature4\model.ckpt
Load pretrained checkpoint: c:\Users\YZM\Desktop\for fun\pytorch\CS480\project\ML-Plant-Properties-Predicion\AutoluonModel\idremovedfeature5\model.ckpt
Load pretrained checkpoint: c:\Users\YZM\Desktop\for fun\pytorch\CS480\project\ML-Plant-Properties-Predicion\AutoluonModel\idremovedfeature6\model.ckpt


In [34]:
prediction1 = predictor1.predict(test_data)
prediction2 = predictor2.predict(test_data)
prediction3 = predictor3.predict(test_data)
prediction4 = predictor4.predict(test_data)
prediction5 = predictor5.predict(test_data)
prediction6 = predictor6.predict(test_data)

Predicting DataLoader 0: 100%|██████████| 200/200 [00:40<00:00,  4.91it/s]


In [35]:
import numpy as np

total_predictions = np.stack((prediction1, prediction2, prediction3, prediction4 , prediction5, prediction6), axis=1)
print(total_predictions.shape)

(6391, 6)


In [36]:
data_frame = pd.read_csv("data/train.csv")

target_columns = data_frame.columns[164:]
infer_min = data_frame[target_columns].min()
infer_max = data_frame[target_columns].max()
print(f"Min values: {infer_min.values}")
print(f"Max values: {infer_max.values}")
print(infer_max.shape)

for i in range(len(total_predictions)):
    total_predictions[i] = total_predictions[i] * (infer_max - infer_min) + infer_min

submission = pd.DataFrame(total_predictions, columns=['X4', 'X11', 'X18', 'X26', 'X50', 'X3112'])
submission['id'] = test_data['id']
submission = submission[['id', 'X4', 'X11', 'X18', 'X26', 'X50', 'X3112']]
submission.to_csv('j228ye_autogluon.csv', index=False)

Min values: [2.92916029e-01 1.33873997e+02 1.96990260e+04 3.45940007e+03
 1.36050911e+01 3.97556994e+05]
Max values: [1.37534090e+00 1.74258658e+02 1.97225418e+04 4.17983799e+03
 1.71674261e+01 4.13183477e+05]
(6,)
